<a href="https://colab.research.google.com/github/fzehrauslu/MakineOgrenmesi/blob/master/Vize_Cevap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')



#model_save_name='sim_autoencoder.pth'
#path = F"/content/gdrive/My Drive/Colab Notebooks/{model_save_name}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
path= F"/content/gdrive/My Drive/Colab Notebooks/arrhythmia.data.txt"
data = pd.read_csv(path, sep=',', header=None)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Columns: 280 entries, 0 to 279
dtypes: float64(120), int64(155), object(5)
memory usage: 988.9+ KB


In [3]:
# veri setindeki ilk 10 sutunu secelim ve onlari ismlendirelim
data1=data.loc[:,0:9]
data1.info()
data1.columns = ["Age", "Sex", "Height", "Weight","QRS duration","P-R interval","Q-T interval","T interval","P interval","QRS"]#,"T","P","QRST","J","Heart Rate"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452 entries, 0 to 451
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       452 non-null    int64
 1   1       452 non-null    int64
 2   2       452 non-null    int64
 3   3       452 non-null    int64
 4   4       452 non-null    int64
 5   5       452 non-null    int64
 6   6       452 non-null    int64
 7   7       452 non-null    int64
 8   8       452 non-null    int64
 9   9       452 non-null    int64
dtypes: int64(10)
memory usage: 35.4 KB


In [4]:
# elde edilen veri setini ilk 5 ornek icin goruntuleyelim
data1.head()

,Age,Sex,Height,Weight,QRS duration,P-R interval,Q-T interval,T interval,P interval,QRS
0,75,0,190,80,91,193,371,174,121,-16
1,56,1,165,64,81,174,401,149,39,25
2,54,0,172,95,138,163,386,185,102,96
3,55,0,175,94,100,202,380,179,143,28
4,75,0,190,80,88,181,360,177,103,-16


In [5]:
import numpy as np
# sinif degerlerini yazdiralim
Classes=data.loc[:,279]
print(Classes)

# 02 ile kodlu sinifa ve 10 ile kodlu sinifa ait hastalari tespit edelim ve onlari uygun siniflarla eslestirelim
Class0_indices=np.where(Classes==2)[0]
Class1_indices=np.where(Classes==10)[0]

0       8
1       6
2      10
3       1
4       7
       ..
447     1
448    10
449     2
450     1
451     1
Name: 279, Length: 452, dtype: int64


In [0]:
# Bu hasta indislerini tek vektor icinde toplayalim. 
SelectedPatientsIndices=np.concatenate((Class0_indices,Class1_indices),0)

In [0]:
# Hedef sinif degerlerini olustur
y=np.concatenate((np.zeros(np.shape(Class0_indices)),np.ones(np.shape(Class1_indices))),0)

#Ilgilendigimiz hastalari  veri setimizi olusturmak icin secelim. 
X = data1.loc[SelectedPatientsIndices,:]
## normalise the dataset
X=X.to_numpy()
X_nanLocations= np.unique(np.where(np.isfinite(X))[0])

X=X[X_nanLocations]
X=np.float64(X)
for i in range(X.shape[1]):
  X[:,i]-=np.mean(X[:,i])
  X[:,i]=X[:,i]/(np.std(X[:,i])+0.0000001)

y=y[X_nanLocations]
from sklearn.model_selection import train_test_split
# Veri setini % 30 test ve %70 training olmak üzere iki kümeye (eğitim ve test kümesi) ayırdik
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

In [16]:
# Logistic Regression modelimizi egitelim
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)
print('Logistic Regression icin accuracy',accuracy_score(y_test, y_pred))

Logistic Regression icin accuracy 0.6896551724137931


In [14]:
# SVM modellerini egit
# lineer kernel, RBF ve Polynomial
#RBF kernel icin gamma degerlerini = 0.1, 0.4 ve 0.7 kullanınız. 
#Polynomial icin degree degerlerini = 1, 3 ve 6 degerleri icin deneyiniz.
from sklearn import svm, datasets
gamma='auto'
from sklearn import svm, datasets
for kernel in ['linear','rbf','poly']:
  for C in [0.0001, 0.4, 10]:
    if kernel=='rbf':
      for gamma in [0.1, 0.4, 0.7]:
        model=svm.SVC(kernel=kernel, gamma=gamma, C=C)
        model.fit(X_train, y_train)
        print('C:',C,',' , 'kernel:', kernel,',' ,'gamma:', gamma, ',' ,'accuracy:', model.score(X_test,y_test))
        
    elif kernel=='poly':
      gamma='auto'
      for degree in [1,3,6]:
        model=svm.SVC(kernel=kernel, degree=degree, gamma=gamma, C=C)
        model.fit(X_train, y_train)
        print('C:',C, ',' ,'kernel:', kernel,',' ,'degree:', degree, ',' ,'accuracy:', model.score(X_test,y_test))
    else:
      model=svm.SVC(kernel=kernel, gamma=gamma, C=C)
      model.fit(X_train, y_train)
      print('C:',C, ',' ,'kernel:', kernel,',','accuracy:', model.score(X_test,y_test))



C: 0.0001 , kernel: linear , accuracy: 0.3448275862068966
C: 0.4 , kernel: linear , accuracy: 0.7241379310344828
C: 10 , kernel: linear , accuracy: 0.6551724137931034
C: 0.0001 , kernel: rbf , gamma: 0.1 , accuracy: 0.3448275862068966
C: 0.0001 , kernel: rbf , gamma: 0.4 , accuracy: 0.3448275862068966
C: 0.0001 , kernel: rbf , gamma: 0.7 , accuracy: 0.3448275862068966
C: 0.4 , kernel: rbf , gamma: 0.1 , accuracy: 0.5862068965517241
C: 0.4 , kernel: rbf , gamma: 0.4 , accuracy: 0.3793103448275862
C: 0.4 , kernel: rbf , gamma: 0.7 , accuracy: 0.3448275862068966
C: 10 , kernel: rbf , gamma: 0.1 , accuracy: 0.5172413793103449
C: 10 , kernel: rbf , gamma: 0.4 , accuracy: 0.4827586206896552
C: 10 , kernel: rbf , gamma: 0.7 , accuracy: 0.4827586206896552
C: 0.0001 , kernel: poly , degree: 1 , accuracy: 0.3448275862068966
C: 0.0001 , kernel: poly , degree: 3 , accuracy: 0.3448275862068966
C: 0.0001 , kernel: poly , degree: 6 , accuracy: 0.3448275862068966
C: 0.4 , kernel: poly , degree: 1 , ac

En yuksek accuracy, 0.7241379310344828 ile lineer SVM modeline ait (C=0.4). Ikinci yuksek deger 0.6896551724137931 ile logistik regresyon modeline ait. 
